# 4.A. Recommender -- Track Level

We'll take time in this notebook to test our track level reccommendation engine leveraging the features generated from audio and lyrical data. This will then be incorporated into a hybrid engine in notebook **4D**

In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import lyricsgenius
import re
import sys
import spotipy
import spotipy.util as util
from sklearn.compose import ColumnTransformer, make_column_transformer
from pyjarowinkler import distance
import nltk

import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import time
import requests
import seaborn as sns
import warnings
import matplotlib.pyplot as plt

import nltk
from nltk.stem import PorterStemmer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB


In [3]:
songs = pd.read_csv('cleaned_with_sentiment_needs_review.csv')

In [4]:
songs.columns

Index(['album', 'artist', 'date', 'features', 'lyrics', 'producers', 'song',
       'artist_clean', 'album_name_clean', 'track_clean', 'acousticness',
       'album_id', 'album_name', 'artist_id', 'danceability', 'duration_ms',
       'energy', 'id', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'preview_url', 'speechiness', 'tempo', 'time_signature',
       'track_href', 'track_id', 'track_name', 'uri', 'valence', 'genres',
       'pop', 'follower', 'cleaned_lyrics', 'split_lyrics', 'number_lines',
       'track_unique_words_pct', 'track_unique_words', 'track_complexity',
       'track_total_words', 'artist_vocab_size', 'artist_vocab_complexity',
       'track_rhyme_density', 'cleaned_lyrics_for_sentiment',
       'sentiment_track_neg', 'sentiment_track_pos', 'sentiment_track_neu',
       'sentiment_track_comp', 'track_sad_words', 'track_angry_words',
       'track_joy_words', 'track_ant_words', 'track_trust_words',
       'track_fear_words', 'track_disgust_words',

In [5]:
songs = songs.drop(columns=['album', 'artist','lyrics','album_id', 'album_name', 'artist_id',
                           'duration_ms', 'id','mode', 'preview_url','time_signature',
                           'track_href', 'track_name', 'uri','split_lyrics',
                           'cleaned_lyrics','cleaned_lyrics_for_sentiment', ])

In [6]:
songs.columns

Index(['date', 'features', 'producers', 'song', 'artist_clean',
       'album_name_clean', 'track_clean', 'acousticness', 'danceability',
       'energy', 'instrumentalness', 'key', 'liveness', 'loudness',
       'speechiness', 'tempo', 'track_id', 'valence', 'genres', 'pop',
       'follower', 'number_lines', 'track_unique_words_pct',
       'track_unique_words', 'track_complexity', 'track_total_words',
       'artist_vocab_size', 'artist_vocab_complexity', 'track_rhyme_density',
       'sentiment_track_neg', 'sentiment_track_pos', 'sentiment_track_neu',
       'sentiment_track_comp', 'track_sad_words', 'track_angry_words',
       'track_joy_words', 'track_ant_words', 'track_trust_words',
       'track_fear_words', 'track_disgust_words', 'track_surprise_words'],
      dtype='object')

In [7]:
pd.DataFrame(songs.producers.str.split(",").str.join(" ")).producers.str.replace("[","").str.replace("]","").str.replace("'","")

0            JUDGE  Kenny Beats
1        Go Grizzly  Pooh Beatz
2                   Kenny Beats
3               DJ On The Track
4           FadedBlackid  Gren8
                  ...          
25125         Beanz N Kornbread
25126                          
25127                          
25128                          
25129                          
Name: producers, Length: 25130, dtype: object

In [8]:
# songs.producers =songs.producers.str.replace(', ', "|").str.replace("'", "").str.replace(" ", "_").str.replace("[", "").str.replace("]", "").str.replace("|", " ")

In [9]:
# songs.producers

In [10]:
songs_for_proc = songs.select_dtypes(exclude='object')

In [11]:
sc = StandardScaler()

In [12]:
songs_sc = sc.fit_transform(songs_for_proc)

In [13]:
pd.DataFrame(songs_sc)

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,0.540329,-0.556579,1.683429,1.817812,-0.180531,1.255334,-0.357073,1.343459,-0.109543,0.592475,...,-0.359876,-2.336612,-0.229534,-0.045619,-0.837469,-0.909611,-0.244932,-0.565102,0.189735,-0.642276
1,0.540329,0.589872,0.779863,-1.913814,-0.196990,-0.095198,-0.470651,-0.950279,1.430821,1.388534,...,0.821809,0.588128,-0.523928,-0.045619,-0.166661,-0.285569,0.214604,-0.281283,-0.561699,-0.489242
2,0.540329,0.456802,1.037031,0.031193,-0.196990,-1.445730,0.017733,0.663794,-0.866330,0.432516,...,0.230966,-2.396916,0.064859,0.506200,-0.725667,-0.909611,1.478330,0.428264,0.565451,-0.336207
3,0.540329,-0.452170,0.008356,-0.906465,-0.196990,0.174908,-0.847160,-0.046622,-0.745780,2.028813,...,0.484185,-0.602875,-0.082338,-0.045619,-1.061071,-1.159227,-1.049121,-0.565102,-0.561699,-0.642276
4,0.540329,-0.841067,0.467090,-1.559025,0.855165,0.715121,0.392540,0.099794,-1.273522,-0.844949,...,0.821809,0.557976,6.688709,0.230291,-0.166661,-0.535186,-0.015164,-0.565102,-0.185982,-0.030138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25125,-1.881046,-0.816424,-1.124577,1.247614,-0.194419,0.445014,-0.533118,0.629199,-1.245394,0.749530,...,-1.457156,0.588128,0.064859,0.644155,-0.613866,-0.535186,-0.474701,0.428264,0.377593,-0.489242
25126,0.539129,0.257197,-0.707546,0.937173,-0.196990,-0.905517,-0.812519,0.559311,-0.976835,0.680316,...,0.568591,-2.351688,-0.376731,0.092336,-0.278462,-0.035953,0.559257,-0.848921,0.753310,-0.183173
25127,-1.881046,1.874512,-0.478179,-0.006820,-0.196990,-1.445730,0.767346,0.576433,0.365961,-1.167449,...,-0.275470,0.557976,1.095236,-0.183574,0.392346,2.959447,0.329488,0.570173,0.189735,0.275931
25128,-1.881046,-0.792011,-1.082874,1.222272,-0.196990,-0.095198,-0.039056,1.155809,0.258805,0.067371,...,-0.022252,0.075544,0.800842,0.092336,0.168743,-0.410378,0.099720,0.570173,0.189735,0.275931


In [14]:
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity 

cosine_similarities = cosine_similarity(songs_sc, songs_sc) 
results = {}
for idx, row in songs.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1] 
    similar_items = [(cosine_similarities[idx][i], songs['track_id'][i]) for i in similar_indices] 
    results[row['track_id']] = similar_items[1:]

In [15]:
def item(id):  
    return songs.loc[songs['track_id'] == id]['track_id'] 
# Just reads the results out of the dictionary.def 
def recommend(item_id, num):   
    recs = results[item_id][:num]   
    for rec in recs: 
        print("Recommended: " + item(rec[1]) + " (score:" +      str(rec[0]) + ")")

In [16]:
recommend('0R7Y3clm1r32PdZD0uSXPD', 10)

4897    Recommended: 0DeKKtNrFALC4nymcFa43g (score:0.8...
Name: track_id, dtype: object
5494    Recommended: 3E9ktwcmAExrLhKal8jClK (score:0.7...
Name: track_id, dtype: object
5201    Recommended: 0Pa8WK67O7lhBfNKsXHXQP (score:0.6...
Name: track_id, dtype: object
5484    Recommended: 6YPJAZl2dbogpLwJEvZWcQ (score:0.6...
Name: track_id, dtype: object
8585    Recommended: 3P7AIvk98T8oVkQIqJZwPe (score:0.6...
Name: track_id, dtype: object
2807    Recommended: 6aEkiWNRLnfFxXuJKEQrvJ (score:0.6...
Name: track_id, dtype: object
8088    Recommended: 1MCl1Tnz75qkjJIBwcZXkz (score:0.6...
Name: track_id, dtype: object
10712    Recommended: 0P6TIFDZtnndIDifIpEeE2 (score:0.6...
Name: track_id, dtype: object
16304    Recommended: 6hkQ6OQ6nhe7QCckH91aGa (score:0.6...
Name: track_id, dtype: object
22971    Recommended: 7qAqDde6Kt2FDta20yyEVV (score:0.6...
Name: track_id, dtype: object


In [17]:
songs.sort_values(by='follower', ascending=False)['track_id']

6226    0OVVOCs6vK5cNl0x9AEpHr
6205    2WC4sK0ryyysQhtDok9Ytr
6198    6DCZcSspjsKoFjzjrWoCdn
6199    3qN5qMTKyEEmiTZD38BNTT
6200    2G7V7zsVDxg1yRsu7Ew9RJ
                 ...          
1137    1fm6Z8tZjSINb76MvyHmcd
1138    3GggeosBkueo5C6ENilbAX
1139    6LwgrDK4xuxMm2bWCt9B2E
1140    20BpQdIjcP080SUQHGH4u3
1131    1gqM6Fe4Q6BA48juO7s8do
Name: track_id, Length: 25130, dtype: object

In [18]:
songs[songs.artist_clean.str.contains('danny')]['track_id']

4893    0R7Y3clm1r32PdZD0uSXPD
4894    4uj4fdTzkXJcQRTHnBhMoC
4895    38Y9OEAzmDy4Su2SIBrKr9
4896    0oNzpFXCsjYXcXzrDXEvNa
4897    0DeKKtNrFALC4nymcFa43g
4898    2gjFvlsyIDKYooHYryvTTY
4899    3kKFM0R62bthgwAXRok9W8
4900    3BmEI0rPWBjR2DUb1JprjF
4901    0tpGKTgHQsIvxcCsrIDogW
4902    58T8UwAv09pn9qUZnS8VvC
4903    0g2Eb9aCaM6YfvzrEysJsY
4904    7nQS7tummIA0c5HkWP1GUB
4905    7kyfBmfFQDuKc5TXKgPrUy
4906    1O3KSlQ6isEld5yki7uq0E
4907    76pvh4YUJdtxyVA37w7Fsp
4908    2dHTQP3IWIuOD6C1uhwGRD
4909    2Q0pt1s7gU5GowQDtaZ4CA
4910    6lVoMmI02u5rFFgexe9f3G
4911    5FxELQREUBIdy6ZRqZ4iuC
4912    2tyX0SSqmcwuGnzM4E1FG6
4913    1iVHnOntYYBQt6SJ5OhLWQ
4914    4ox8bpBggBqCQv56b4XXTf
4915    4HFsQRxQLEPleq9UmIzke4
4916    00fbJE5O1Z1RmY81c9qAMv
4917    0rFX6pVvOkXvDgiTR5v5kX
4918    6L33UVo5duBnDceNcQXKTW
4919    7fgUmQ0mXB3TjTKYgZix0S
4920    5B7N2FqkVSG5ChTupAxVb4
4921    6BAKNnYzWjE7OYlWanmRDq
4922    5FKaV2J0xiwRENDsl0bMn4
4923    6BugxvxjNZjR512ZbsM4cz
4924    29Zj1r8cek15xe08vNUOLy
4925    

In [19]:
songs[(songs['track_id'] == '0R7Y3clm1r32PdZD0uSXPD') | (songs['track_id'] == '3LVxoI7imdEnGi5BiDde4Z')]

,date,features,producers,song,artist_clean,album_name_clean,track_clean,acousticness,danceability,energy,...,sentiment_track_neu,sentiment_track_comp,track_sad_words,track_angry_words,track_joy_words,track_ant_words,track_trust_words,track_fear_words,track_disgust_words,track_surprise_words
3126,2015.0,[],['Thomas Mellor'],Pain,bugzy malone,walk with me,pain,0.1000,0.383,0.770,...,0.43,-1.00,101,73,4,4,9,93,115,4
4893,2016.0,[],['Paul White'],AinÃ¢â¬â¢t it Funny,danny brown,atrocity exhibition,aint it funny,0.0108,0.483,0.841,...,0.78,0.95,8,8,8,9,4,12,8,3
